## Create Argonaut Patient Resource Using the Python FHIRClient Module

### Prerequisites:

- Python 3.6 or greater
    for instruction how to install Python and modules and optionally setting up atom as a python IDE and creating virtual environments, If you are new to Python or need help getting starged I recommend these YouYube videos by Corey Schaefer:
    
[![image.jpg](http://img.youtube.com/vi/YYXdXT2l-Gg/sddefault.jpg)](https://www.youtube.com/watch?v=YYXdXT2l-Gg)

[![image.jpg](http://img.youtube.com/vi/CqvZ3vGoGs0/sddefault.jpg)](https://www.youtube.com/watch?v=CqvZ3vGoGs0)

[![image.jpg](http://img.youtube.com/vi/DjEuROpsvp4/sddefault.jpg)](https://www.youtube.com/watch?v=DjEuROpsvp4)

[![image.jpg](http://img.youtube.com/vi/zDYL22QNiWk/sddefault.jpg)](https://www.youtube.com/watch?v=zDYL22QNiWk)


- installed modules:
   - [fhirclient](https://github.com/smart-on-fhir/client-py)  ( FHIR version STU3 )
   - [requests](docs.python-requests.org/en/master/)
   - [csv — CSV File Reading and Writing](https://docs.python.org/3/library/csv.html)
   
### Introduction:
 
IN the code below, I walk through the steps to create an Argonaut (FHIR DSTU2) Patient resource, validate it, Post it to a FHIR server and then fetch it in xml format.  The data is based off of a simple excel file which serves as a data entry form.  To simplify things the worksheet is saved as a CSV file which is used directly in the code below.  

See this [nice introduction to the FHIRClient documentation](https://github.com/bmilius-nmdp/pytestfhir/blob/master/MakePatSpeObs.ipynb) by Bob Milius for a quick overview how to use the fhirclient models.  The models are instantiated with a Python dictionary structure or primitives types ( str, int, float, or bool ).   Repeating elements are represented at lists.  Also the FHIRClient models perform some data validation as well and provide for a more `pythonic` way to represent FHIR resources.
 

#### First import all the necessary modules

We will be using all these FHIRClient models ( classes ) for this examples and they include the patient resource models and all the datatype classes used by the resource.  The `IPython` is used to make this Jupyter Notebook display pretty.

The version of the fhirclient.models used for DSTU2 Resource is 1.0.3  ( see the Fhirclient docs )

In [9]:
# ******************** modules **************************

import fhirclient.models.patient as P
import fhirclient.models.identifier as I
import fhirclient.models.coding as C
import fhirclient.models.codeableconcept as CC
import fhirclient.models.fhirdate as D
import fhirclient.models.extension as X
import fhirclient.models.humanname as H
import fhirclient.models.meta as M
import fhirclient.models.valueset as VS
from json import dumps
from requests import get, post, put
import os
from csv import DictReader
from IPython.display import display, Markdown
from pprint import pprint


####  Assign Global Variables

Here is where we assign all the global varaibles for this example such as the local paths for file input and output which need to
be updated for your local environment.

In [23]:
out_path = '/Users/ehaas/Documents/FHIR/pyfhir/test/'
#out_path = "C:/Users/Eric/Documents/Jan_2019_FHIR_Experience"
fhir_term_server = 'http://fhirtest.uhn.ca/baseDstu2'
fhir_test_server = 'http://fhirtest.uhn.ca/baseDstu2'

headers = {
'Accept':'application/fhir+json',
'Content-Type':'application/fhir+json'
}

# profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient
profile = 'http://fhir.org/guides/argonaut/StructureDefinition/argo-patient'

code_map = {  # usign o local hash map to store displays since unable to locate open test terminology server that support the argo valuesets lookups :-(
'ar': 'Arabic',
'bn': 'Bengali',
'cs': 'Czech',
'da': 'Danish',
'de': 'German',
'de-AT': 'German (Austria)',
'de-CH': 'German (Switzerland)',
'de-DE': 'German (Germany)',
'el': 'Greek',
'en': 'English',
'en-AU': 'English (Australia)',
'en-CA': 'English (Canada)',
'en-GB': 'English (Great Britain)',
'en-IN': 'English (India)',
'en-NZ': 'English (New Zeland)',
'en-SG': 'English (Singapore)',
'en-US': 'English (United States)',
'es': 'Spanish',
'es-AR': 'Spanish (Argentina)',
'es-ES': 'Spanish (Spain)',
'es-UY': 'Spanish (Uruguay)',
'fi': 'Finnish',
'fr': 'French',
'fr-BE': 'French (Belgium)',
'fr-CH': 'French (Switzerland)',
'fr-FR': 'French (France)',
'fy': 'Frysian',
'fy-NL': 'Frysian (Netherlands)',
'hi': 'Hindi',
'hr': 'Croatian',
'it': 'Italian',
'it-CH': 'Italian (Switzerland)',
'it-IT': 'Italian (Italy)',
'ja': 'Japanese',
'ko': 'Korean',
'nl': 'Dutch',
'nl-BE': 'Dutch (Belgium)',
'nl-NL': 'Dutch (Netherlands)',
'no': 'Norwegian',
'no-NO': 'Norwegian (Norway)',
'pa': 'Punjabi',
'pl': 'Polish',
'pt': 'Portuguese',
'pt-BR': 'Portuguese (Brazil)',
'ru': 'Russian',
'ru-RU': 'Russian (Russia)',
'sr': 'Serbian',
'sr-RS': 'Serbian (Serbia)',
'sv': 'Swedish',
'sv-SE': 'Swedish (Sweden)',
'te': 'Telegu',
'zh': 'Chinese',
'zh-CN': 'Chinese (China)',
'zh-HK': 'Chinese (Hong Kong)',
'zh-SG': 'Chinese (Singapore)',
'zh-TW': 'Chinese (Taiwan)',
'1002-5': 'American Indian or Alaska Native',
'2028-9': 'Asian',
'2054-5': 'Black or African American',
'2076-8': 'Native Hawaiian or Other Pacific Islander',
'2106-3': 'White',
'2135-2': 'Hispanic or Latino',
'2186-5': 'Non Hispanic or Latino'
}


####  Instanatiating All the Classes

Before we start assigning patient data to the Patient Class, The classes are instantatiated.  See [this video](https://www.youtube.com/watch?v=ZDa-Z5JzLYM) for more
information about Python classes and instances.  For the FHIRClient Python models, the classes are represented as Python dictionary structures using the method `as_json()`

In [5]:
#********************* instanatiate classes **********************

patient = P.Patient()
patientcommunication = P.PatientCommunication()
meta = M.Meta()
identifier = I.Identifier()
name = H.HumanName()
birthdate = D.FHIRDate()
language = CC.CodeableConcept()
coding = C.Coding()
race = C.Coding()
ethn = C.Coding()
b_sex_extension = X.Extension()
race_extension = X.Extension()
race_ombcat_extension = X.Extension()
race_ombdet_extension = X.Extension()
race_text_extension = X.Extension()
ethn_extension = X.Extension()
ethn_ombcat_extension = X.Extension()
ethn_ombdet_extension = X.Extension()
ethn_text_extension = X.Extension()


display(Markdown(f'**Instantiated Python FHIR Patient Class as a Python dictionary object:** `{patient.as_json()}`'))


**Instantiated Python FHIR Patient Class as a Python dictionary object:** `{'resourceType': 'Patient'}`

#### Get patient data

To populate the patient resource, the data is entered from some source.  The datasource may be a database, spreadsheet, manual entry
(e.g. from a form). For this example, a simple Python dictionary file of name:value pairs is used.  See the US Core Patient example for how daa from a csv file is imported into the resource.

In [6]:
data = dict(
mrn = '1234',
f_name = 'Eric',
l_name = "Haas",
mi = 'M',
b_date = '1964-06-19',
gender = 'male',
b_sex = 'M',
lang = 'en',
race = '2106-3', # code for 'White'
ethn = '2186-5', # Not hispanic or latino
)


#### Create Patient resource

After getting the source data, the patient resource can be populated.   The following code cells shows step by step how the different data types are assigned using the FHIRClient model.

#####  Assign values to the simple id and active elements

In [10]:
# create simple id and active elements
patient.id = 'test-1'
patient.active = True
pprint(patient.as_json())

{'active': True, 'id': 'test-1', 'resourceType': 'Patient'}


#####  Assign values to complex `meta` element 
 
 The meta class contains a repeating `profile` elment

In [11]:
meta.profile = []
meta.profile.append(profile)
patient.meta = meta
pprint(patient.as_json())

{'active': True,
 'id': 'test-1',
 'meta': {'profile': ['http://fhir.org/guides/argonaut/StructureDefinition/argo-patient']},
 'resourceType': 'Patient'}


#####  Assign values to the complex identifier element

Note in this example only a identifier is provided.

In [12]:
# create complex identifier element
identifier.system = 'http://example.org/pids'
identifier.value = data['mrn']
patient.identifier=[]
patient.identifier.append(identifier)
pprint(patient.as_json())

{'active': True,
 'id': 'test-1',
 'identifier': [{'system': 'http://example.org/pids', 'value': '1234'}],
 'meta': {'profile': ['http://fhir.org/guides/argonaut/StructureDefinition/argo-patient']},
 'resourceType': 'Patient'}


#####  Assign values to the complex repeating name element

Note in this example only a single name is provided.

In [13]:
# create complex repeating name element
name.family = []
name.family.append(data['l_name'])
name.given = []
name.given.append(data['f_name'])
if data['mi']: name.given.append(data['mi'])
patient.name=[]
patient.name.append(name)
pprint(patient.as_json())

{'active': True,
 'id': 'test-1',
 'identifier': [{'system': 'http://example.org/pids', 'value': '1234'}],
 'meta': {'profile': ['http://fhir.org/guides/argonaut/StructureDefinition/argo-patient']},
 'name': [{'family': ['Haas'], 'given': ['Eric', 'M']}],
 'resourceType': 'Patient'}


#####   Assign simple `gender` and `birthdate` elements

Birthdate is treated as an iso string in Python.

In [14]:
#create simple gender and birthdate element ( birthdate is treated as iso string in Python)
patient.gender = data['gender']
b_date = D.FHIRDate(data['b_date'])
patient.birthDate = b_date
pprint(patient.as_json())


{'active': True,
 'birthDate': '1964-06-19',
 'gender': 'male',
 'id': 'test-1',
 'identifier': [{'system': 'http://example.org/pids', 'value': '1234'}],
 'meta': {'profile': ['http://fhir.org/guides/argonaut/StructureDefinition/argo-patient']},
 'name': [{'family': ['Haas'], 'given': ['Eric', 'M']}],
 'resourceType': 'Patient'}


#####  Assign backbone `communication` element 

`CodeableConcept` and `Coding` datatypes are used in this element.  This pattern is repeated for other coded elements below.

In [15]:
# create backbone communication element with codeableconcept
patient.communication = []
dis = code_map[data['lang']]
sys = 'urn:ietf:bcp:47'
language.coding = []
coding.code = data['lang']
coding.system = sys
coding.display = dis
language.coding.append(coding)
language.text = dis
patientcommunication.language = language
patient.communication.append(patientcommunication)
pprint(patient.as_json())

{'active': True,
 'birthDate': '1964-06-19',
 'communication': [{'language': {'coding': [{'code': 'en',
                                             'display': 'English',
                                             'system': 'urn:ietf:bcp:47'}],
                                 'text': 'English'}}],
 'gender': 'male',
 'id': 'test-1',
 'identifier': [{'system': 'http://example.org/pids', 'value': '1234'}],
 'meta': {'profile': ['http://fhir.org/guides/argonaut/StructureDefinition/argo-patient']},
 'name': [{'family': ['Haas'], 'given': ['Eric', 'M']}],
 'resourceType': 'Patient'}


#####  Assign values to the extension

There are three extensions defined for the Argonaut Patient.  The "Birth Sex" extension is a simple extension and the "Race" and "Ethnicity" extensions are complex extensions.

######  Assign values to the simple "Birth Sex" extension

For each simple extension, a url and a value ( in this case a code ) are assigned.

In [21]:
# create simple birth sex (repeating) extension
patient.extension = []
b_sex_extension.url = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-birthsex'
b_sex_extension.url = 'http://fhir.org/guides/argonaut/StructureDefinition/argo-birthsex'  # canonical url for birthsex is 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-birthsex'
b_sex_extension.valueCode = data['b_sex']
patient.extension.append(b_sex_extension)
pprint(patient.as_json())

{'active': True,
 'birthDate': '1964-06-19',
 'communication': [{'language': {'coding': [{'code': 'en',
                                             'display': 'English',
                                             'system': 'urn:ietf:bcp:47'}],
                                 'text': 'English'}}],
 'extension': [{'url': 'http://fhir.org/guides/argonaut/StructureDefinition/argo-birthsex',
                'valueCode': 'M'}],
 'gender': 'male',
 'id': 'test-1',
 'identifier': [{'system': 'http://example.org/pids', 'value': '1234'}],
 'meta': {'profile': ['http://fhir.org/guides/argonaut/StructureDefinition/argo-patient']},
 'name': [{'family': ['Haas'], 'given': ['Eric', 'M']}],
 'resourceType': 'Patient'}


###### Assign values to the complex "race" extension

For each complex extension, a url and nested extensions and a values ( in this case a CodeableConcepts and strings ) are assigned.

For this example, a simple hash map is used to fetch a code display for the race code.  For an example of using a Terminology Server and the $lookup operation, see the US Core Patient example.

In [24]:
# create complex race  (repeating) extension using codeableconcept

sys = 'http://hl7.org/fhir/v3/Race'  if data['race'] not in ['UNK','ASKU'] else 'http://hl7.org/fhir/v3/NullFlavor'
dis = code_map[data['race']]
race_extension.url ='http://fhir.org/guides/argonaut/StructureDefinition/argo-race'
# race_extension.url = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-race'  # canonical url for race
race_extension.extension = []
race_text_extension.url = 'text'  # url for "subextension"
race_text_extension.valueString = dis  # use display
race_extension.extension.append(race_text_extension)

race_ombcat_extension.url = 'ombCategory'  # url for "subextension"
race.system = sys  # look up using terminology server
race.code = data['race']
race.display = dis  # look up using terminology server
race_ombcat_extension.valueCoding = race
race_extension.extension.append(race_ombcat_extension)

patient.extension.append(race_extension)
pprint(patient.as_json())


{'active': True,
 'birthDate': '1964-06-19',
 'communication': [{'language': {'coding': [{'code': 'en',
                                             'display': 'English',
                                             'system': 'urn:ietf:bcp:47'}],
                                 'text': 'English'}}],
 'extension': [{'url': 'http://fhir.org/guides/argonaut/StructureDefinition/argo-birthsex',
                'valueCode': 'M'},
               {'extension': [{'url': 'text', 'valueString': 'White'},
                              {'url': 'ombCategory',
                               'valueCoding': {'code': '2106-3',
                                               'display': 'White',
                                               'system': 'http://hl7.org/fhir/v3/Race'}}],
                'url': 'http://fhir.org/guides/argonaut/StructureDefinition/argo-race'}],
 'gender': 'male',
 'id': 'test-1',
 'identifier': [{'system': 'http://example.org/pids', 'value': '1234'}],
 'meta': {'profile': ['ht

Finally to complete this example , the ehtnicity extension is populated with a similar pattern to the race extension above

In [25]:
# create complex ethnicity (repeating) extension using codeableconcept
sys = 'http://hl7.org/fhir/ValueSet/v3-Ethnicity'
dis = code_map[data['ethn']]

ethn_extension.url = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity'  # canonical url for ethn
ethn_extension.extension = []

ethn_text_extension.url = 'text'  # url for "subextension"
ethn_text_extension.valueString = dis  # use display
ethn_extension.extension.append(ethn_text_extension)

ethn_ombcat_extension.url = 'ombCategory'  # url for "subextension"
ethn.system = sys  # look up using terminology server
ethn.code = data['ethn']
ethn.display = dis  # look up using terminology server
ethn_ombcat_extension.valueCoding = ethn
ethn_extension.extension.append(ethn_ombcat_extension)

patient.extension.append(ethn_extension)
pprint(patient.as_json())


{'active': True,
 'birthDate': '1964-06-19',
 'communication': [{'language': {'coding': [{'code': 'en',
                                             'display': 'English',
                                             'system': 'urn:ietf:bcp:47'}],
                                 'text': 'English'}}],
 'extension': [{'url': 'http://fhir.org/guides/argonaut/StructureDefinition/argo-birthsex',
                'valueCode': 'M'},
               {'extension': [{'url': 'text', 'valueString': 'White'},
                              {'url': 'ombCategory',
                               'valueCoding': {'code': '2106-3',
                                               'display': 'White',
                                               'system': 'http://hl7.org/fhir/v3/Race'}}],
                'url': 'http://fhir.org/guides/argonaut/StructureDefinition/argo-race'},
               {'extension': [{'url': 'text',
                               'valueString': 'Non Hispanic or Latino'},
                

#### Pretty Print the resource as a json string

In Python the resource is represented as a 'dict' type.  to share this data it needs to be converted to JSON format using the Python standard library methods from the "json" module

In [27]:
# pretty print the resource as a json string  ( in Python is a 'dict' type)

print('\n',dumps(patient.as_json(),indent =3 ))



 {
   "id": "test-1",
   "meta": {
      "profile": [
         "http://fhir.org/guides/argonaut/StructureDefinition/argo-patient"
      ]
   },
   "extension": [
      {
         "url": "http://fhir.org/guides/argonaut/StructureDefinition/argo-birthsex",
         "valueCode": "M"
      },
      {
         "extension": [
            {
               "url": "text",
               "valueString": "White"
            },
            {
               "url": "ombCategory",
               "valueCoding": {
                  "code": "2106-3",
                  "display": "White",
                  "system": "http://hl7.org/fhir/v3/Race"
               }
            }
         ],
         "url": "http://fhir.org/guides/argonaut/StructureDefinition/argo-race"
      },
      {
         "extension": [
            {
               "url": "text",
               "valueString": "Non Hispanic or Latino"
            },
            {
               "url": "ombCategory",
               "valueCoding": {
    

####  Validate the Resource

Using the [$validate](http://build.fhir.org/resource-operation-validate.html) operation, the example is validated 
by a FHIR Reference Server.  The results are displayed below in the human readable text as xhtml.

**NOTE** Unable to locate a Server that performs the \$validate operation on the Argonaut Profiles.  Therefore this operation fails.  See the US Core Patient example for a successful outcome using this operation.

In [28]:
# *********************** validate Resource ********************************

params = {
'profile': profile
}
#   r = requests.post('https://httpbin.org/post', data = {'key':'value'})
r = post(f'{fhir_test_server}/Patient/$validate', params = params, headers = headers, data = dumps(patient.as_json()))
# print(r.status_code)
# view  output
display(Markdown(f'<h1>Validation output</h1>{r.json()["text"]["div"]}'))


<h1>Validation output</h1><div xmlns="http://www.w3.org/1999/xhtml"><h1>Operation Outcome</h1><table border="0"><tr><td style="font-weight: bold;">error</td><td>[]</td><td><pre>Unexpected failure while validating resource</pre></td>
					
				
			</tr>
		</table>
	</div>

##### PUT Resource on FHIR Server

Using the FHIR RESTful `PUT` transaction, the resource is loaded onto the FHIR test server. The results are displayed below in the human readable text as xhtml.

In [29]:
# *********************** PUT Resource on server********************************

#   r = requests.post('https://httpbin.org/post', data = {'key':'value'})
r = put(f'{fhir_test_server}/Patient/{patient.id}', headers = headers, data = dumps(patient.as_json()))

display(Markdown(f'<h1>Put Response</h1><p><strong>Transaction Status</strong> = {r.status_code}</p>{r.json()["text"]["div"]}<p>raw output :</p><code>{dumps(r.json(),indent=3)}</code>'))


<h1>Put Response</h1><p><strong>Transaction Status</strong> = 200</p><div xmlns="http://www.w3.org/1999/xhtml"><div class="hapiHeaderText">Eric M <b>HAAS </b></div><table class="hapiPropertyTable"><tbody><tr><td>Identifier</td><td>1234</td></tr><tr><td>Date of birth</td><td><span>19 June 1964</span></td></tr></tbody></table></div><p>raw output :</p><code>{
   "resourceType": "Patient",
   "id": "test-1",
   "meta": {
      "versionId": "1",
      "lastUpdated": "2019-01-10T20:07:31.892+00:00",
      "profile": [
         "http://fhir.org/guides/argonaut/StructureDefinition/argo-patient"
      ]
   },
   "text": {
      "status": "generated",
      "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><div class=\"hapiHeaderText\">Eric M <b>HAAS </b></div><table class=\"hapiPropertyTable\"><tbody><tr><td>Identifier</td><td>1234</td></tr><tr><td>Date of birth</td><td><span>19 June 1964</span></td></tr></tbody></table></div>"
   },
   "extension": [
      {
         "url": "http://fhir.org/guides/argonaut/StructureDefinition/argo-birthsex",
         "valueCode": "M"
      },
      {
         "url": "http://fhir.org/guides/argonaut/StructureDefinition/argo-race",
         "extension": [
            {
               "url": "text",
               "valueString": "White"
            },
            {
               "url": "ombCategory",
               "valueCoding": {
                  "system": "http://hl7.org/fhir/v3/Race",
                  "code": "2106-3",
                  "display": "White"
               }
            }
         ]
      },
      {
         "url": "http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity",
         "extension": [
            {
               "url": "text",
               "valueString": "Non Hispanic or Latino"
            },
            {
               "url": "ombCategory",
               "valueCoding": {
                  "system": "http://hl7.org/fhir/ValueSet/v3-Ethnicity",
                  "code": "2186-5",
                  "display": "Non Hispanic or Latino"
               }
            }
         ]
      }
   ],
   "identifier": [
      {
         "system": "http://example.org/pids",
         "value": "1234"
      }
   ],
   "active": true,
   "name": [
      {
         "family": [
            "Haas"
         ],
         "given": [
            "Eric",
            "M"
         ]
      }
   ],
   "gender": "male",
   "birthDate": "1964-06-19",
   "communication": [
      {
         "language": {
            "coding": [
               {
                  "system": "urn:ietf:bcp:47",
                  "code": "en",
                  "display": "English"
               }
            ],
            "text": "English"
         }
      }
   ]
}</code>

##### GET Resource from FHIR Server in XML format

Using the FHIR RESTful `GET` transaction, fetch the resource from the FHIR test server. The results are displayed below  human  xml.

In [30]:
# *********************** GET Resource as xml from server********************************
headers_xml = {
'Accept':'application/fhir+xml',
'Content-Type':'application/fhir+xml'
}

#   r = requests.post('https://httpbin.org/post', data = {'key':'value'})
r = get(f'{fhir_test_server}/Patient/{patient.id}', headers = headers_xml)

display(Markdown(f'<h1>Get Resource as XML</h1><p><strong>Transaction Status</strong> = {r.status_code}</p>'))
print(r.text)

<h1>Get Resource as XML</h1><p><strong>Transaction Status</strong> = 200</p>

<Patient xmlns="http://hl7.org/fhir">
   <id value="test-1"/>
   <meta>
      <versionId value="1"/>
      <lastUpdated value="2019-01-10T20:07:31.892+00:00"/>
      <profile value="http://fhir.org/guides/argonaut/StructureDefinition/argo-patient"/>
   </meta>
   <text>
      <status value="generated"/>
      <div xmlns="http://www.w3.org/1999/xhtml">
         <div class="hapiHeaderText">Eric M 
            <b>HAAS </b>
         </div>
         <table class="hapiPropertyTable">
            <tbody>
               <tr>
                  <td>Identifier</td>
                  <td>1234</td>
               </tr>
               <tr>
                  <td>Date of birth</td>
                  <td>
                     <span>19 June 1964</span>
                  </td>
               </tr>
            </tbody>
         </table>
      </div>
   </text>
   <extension url="http://fhir.org/guides/argonaut/StructureDefinition/argo-birthsex">
      <valueCode value="M"/>
   </extension>
   <extension u

### Extending this example

There are several ways to improve the above code:

For example

- Getting data from remote source
- Using a terminology server for code display and lookup
- Checking for missing data
- adding capability for repeating elements
- refactoring code to make more compact and reusable

## FIN